# Environment Setup

In [ ]:
import math
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

# Python version
assert sys.version_info >= (3, 8)
# Open3D version
assert o3d.__version__ >= "0.17.0"

In [ ]:
# Point cloud file I/O
filename = "data/input/iScan-Pcd-1-1.ply"
base_name, extension = os.path.splitext(os.path.basename(filename))

output_path = "data/output/" + base_name
os.makedirs(output_path, exist_ok=True)

# 1. Point Cloud Filtering

## Step 1. Down-sampling

In [ ]:
def down_sample(input_filename, output_filename, every_k_points=10):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Function uniformly down-samples the point cloud, evenly select 1 point for every k points
    down_sampled_pcd = pcd.uniform_down_sample(every_k_points)

    # Save the down-sampled point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, down_sampled_pcd, write_ascii=False)

In [ ]:
# Perform down sampling
down_sampled_filename = os.path.join(output_path, base_name + " - downsampled" + extension)
down_sample(filename, down_sampled_filename)

## Step 2. Outlier removal

In [ ]:
# Input point cloud file
down_sampled_filename = os.path.join(output_path, base_name + " - downsampled" + extension)

In [ ]:
def outlier_removal(input_filename, output_filename, nb_neighbors=20, std_ratio=2.0):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Statistical outlier removal
    filtered_pcd, mask = pcd.remove_statistical_outliers(nb_neighbors, std_ratio)

    # Color outliers in red
    outlier = pcd.select_by_mask(mask, invert=True)
    outlier.paint_uniform_color([1.0, 0.0, 0.0])
    print(f"Remove {outlier.point.positions.shape[0]} outliers.")

    # Save the outlier for comparison
    outlier_filename = os.path.join(output_path, base_name + " - outlier" + extension)
    o3d.t.io.write_point_cloud(outlier_filename, outlier, write_ascii=False)

    # Save the filtered point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, filtered_pcd, write_ascii=False)

In [ ]:
# Perform outlier removal
filtered_filename = os.path.join(output_path, base_name + " - filtered" + extension)
outlier_removal(down_sampled_filename, filtered_filename)

# 2. Point Cloud Segmentation

## Step 1. DBSCAN clustering

In [ ]:
# Input point cloud file
filtered_filename = os.path.join(output_path, base_name + " - filtered" + extension)

In [ ]:
from collections import Counter


def dbscan_clustering(input_filename, output_filename, eps=0.10, min_points=5):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)
    print(pcd.point, '\n')

    # Mask by intensity to acquire low intensity points
    intensity_threshold = 50
    mask = np.where(pcd.point.intensity.numpy() <= intensity_threshold, True, False).flatten()

    # Convert mask to index
    index_of_mask = [i for i, boolean_val in enumerate(mask) if boolean_val]

    # Extract low intensity points, then perform DBSCAN on them
    low_intensity_pcd = pcd.select_by_mask(mask)
    labels = low_intensity_pcd.cluster_dbscan(eps, min_points)
    labels = labels.numpy()

    n_clusters = labels.max() + 1
    print(f"DBSCAN clustering return {n_clusters} clusters.\n")

    # Clusters will be labeled in a way that cluster with the most points is labeled 1
    if n_clusters >= 2:
        counter = Counter(labels)

        # Remove noise that be labeled -1
        del counter[-1]

        # Sort by count
        sorted_items = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        rank = [item[0] for item in sorted_items]

        # Add cluster attribute to the input point cloud
        cluster = np.zeros(len(mask)).astype(np.int32)
        for index, val in enumerate(labels):
            if val != -1:
                cluster[index_of_mask[index]] = rank.index(val) + 1
        pcd.point.cluster = np.reshape(cluster, (len(cluster), 1))
        print(pcd.point)

        # Save the clustered point cloud to an output file
        o3d.t.io.write_point_cloud(output_filename, pcd, write_ascii=False)

    else:
        print("Warning: DBSCAN clustering return less than 2 clusters.")

In [ ]:
# Perform DBSCAN clustering
clustered_filename = os.path.join(output_path, base_name + " - clustered" + extension)
dbscan_clustering(filtered_filename, clustered_filename)

## Step 2. Curve fitting

In [ ]:
# Input point cloud file
clustered_filename = os.path.join(output_path, base_name + " - clustered" + extension)

In [ ]:
from scipy import interpolate
from scipy.spatial import cKDTree

# B-spline representation of centre curve
curve_point_centre = []


def curve_fitting(input_filename):
    global curve_point_centre

    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Step 1. Fit two curves on both left and right
    # Use cluster attribute to extrack rail points
    cluster = pcd.point.cluster.numpy()
    left_rail_mask = (cluster == 1).flatten()
    right_rail_mask = (cluster == 2).flatten()

    left_rail = pcd.select_by_mask(left_rail_mask)
    point_left = left_rail.point.positions.numpy()
    point_num_left = point_left.shape[0]
    print("Left rail points:", point_num_left)

    right_rail = pcd.select_by_mask(right_rail_mask)
    point_right = right_rail.point.positions.numpy()
    point_num_right = point_right.shape[0]
    print("Right rail points:", point_num_right, '\n')

    scalar_factor = point_num_left / point_num_right
    print("scalar_factor:", scalar_factor, '\n')

    point_left = np.transpose(point_left)  # (n, 3) -> (3, n)
    point_right = np.transpose(point_right)

    # Find the B-spline representation of an 3-D curve
    tck_left, _ = interpolate.splprep(point_left)
    tck_right, _ = interpolate.splprep(point_right)

    u_left = np.linspace(0, 1, point_num_left)
    knots_left = interpolate.splev(u_left, tck_left)
    curve_point_left = np.column_stack((knots_left[0], knots_left[1], knots_left[2]))

    u_right = np.linspace(0, 1, point_num_right)
    knots_right = interpolate.splev(u_right, tck_right)
    curve_point_right = np.column_stack((knots_right[0], knots_right[1], knots_right[2]))

    # Step 2. Calculate the centre line, then fit a curve on it
    # Build a kd-tree from the left curve points
    kdtree = cKDTree(curve_point_left)

    # Query the kd-tree to find the nearest neighbor and its distance
    distance, nearest_index = kdtree.query(curve_point_right)

    # Get the nearest point from the left curve points
    nearest_point = curve_point_left[nearest_index]

    # Points on the centre line
    point_centre = (nearest_point + curve_point_right) / 2
    point_centre = np.transpose(point_centre)  # (n, 3) -> (3, n)

    # Fit a curve on centre line
    tck_centre, _ = interpolate.splprep(point_centre)

    u_centre = np.linspace(0, 1, point_num_right)
    knots_centre = interpolate.splev(u_centre, tck_centre)
    curve_point_centre = np.column_stack((knots_centre[0], knots_centre[1], knots_centre[2]))

    # Step 3. Store three fitted curves to a separate file
    point = np.vstack((curve_point_left, curve_point_right, curve_point_centre))

    colors = np.vstack((np.full((curve_point_left.shape[0], 3), [0.0, 0.0, 1.0]),  # color in blue
                        np.full((curve_point_right.shape[0], 3), [0.0, 1.0, 0.0]),  # color in green
                        np.full((curve_point_centre.shape[0], 3), [1.0, 0.0, 0.0])))  # color in red

    curve = np.vstack((np.full((curve_point_left.shape[0], 1), 1),
                       np.full((curve_point_right.shape[0], 1), 2),
                       np.full((curve_point_centre.shape[0], 1), 3))).astype(np.uint8)

    pcd_curve = o3d.t.geometry.PointCloud()
    pcd_curve.point.positions = point
    pcd_curve.point.colors = colors
    pcd_curve.point.curve = curve

    # Save the fitted curves for comparison
    curve_filename = os.path.join(output_path, base_name + " - curve" + extension)
    o3d.t.io.write_point_cloud(curve_filename, pcd_curve, write_ascii=False)

    return tck_centre, u_centre

In [ ]:
tck_curve, u_curve = curve_fitting(clustered_filename)  # tck, u
print("Centre curve parameters:", end=" ")
print(f"tck_curve[0] - t: {tck_curve[0]}\n")
print(f"tck_curve[1] - c: {np.array(tck_curve[1])}\n")
print(f"tck_curve[2] - k: {tck_curve[2]}\n")
print(f"u_curve: {len(u_curve)} {u_curve}")

## 3. Cross-Section Extraction

In [ ]:
def segment_ground(input_filename, distance_threshold=0.05, ransac_n=3, num_iterations=100):
    pcd = o3d.t.io.read_point_cloud(input_filename)
    plane_model, _ = pcd.segment_plane(distance_threshold, ransac_n, num_iterations)
    return plane_model

In [ ]:
a, b, c, d = plane = segment_ground(clustered_filename).numpy()  # ax + by + cz + d = 0
print("plane parameters:", plane)

## Step 1. Coordinate Transformation

In [ ]:
# Calculate coordinates in the new coordinate system
def coordinate_transformation(input_filename, output_filename):
    global curve_point_centre

    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Coordinates in original coordinate system
    coordinates = pcd.point.positions.numpy()

    # Prepare for coordinate calculation
    normal_z = np.array([a, b, c])
    if np.dot(normal_z, curve_point_centre[0]) + d < 0:  # centre curve points should have z > 0
        normal_z = -normal_z
    magnitude_z = np.linalg.norm(normal_z)

    diff = np.diff(curve_point_centre, axis=0)
    diff = np.insert(diff, 0, [0., 0., 0.], axis=0)
    distances = np.linalg.norm(diff, axis=1)
    cumulate_y = np.cumsum(distances)

    print(f"curve_point_centre: {curve_point_centre.shape}\n{curve_point_centre[:3]}\n")
    print(f"diff: {diff.shape}\n{diff[:3]}\n")
    print(f"distances: {distances.shape}\n{distances[:3]}\n")
    print(f"cumulate_y: {cumulate_y.shape}\n{cumulate_y[:3]}\n")

    kdtree_centre = cKDTree(curve_point_centre)
    distance, nearest_index = kdtree_centre.query(coordinates)
    nearest_point = curve_point_centre[nearest_index]

    PQ = coordinates - nearest_point
    magnitude_PQ = distance
    cross_product = np.cross(normal_z, PQ)
    sin_theta = np.linalg.norm(cross_product, axis=1) / (magnitude_z * magnitude_PQ)

    normal_y = diff
    dot_product = np.sum(cross_product * normal_y[nearest_index], axis=1)
    sign_x = np.where(dot_product > 0, 1, -1)

    print(f"cross_product: {cross_product.shape}\n{cross_product[:3]}\n")
    print(f"normal_y: {normal_y.shape}\n{normal_y[:3]}\n")
    print(f"dot_product: {dot_product.shape}\n{dot_product[:3]}\n")

    # Coordinate calculation
    x = magnitude_PQ * sin_theta * sign_x
    y = cumulate_y[nearest_index]
    z = (np.dot(coordinates, normal_z) + d) / magnitude_z

    pcd.point.positions = np.column_stack((x, y, z))

    o3d.t.io.write_point_cloud(output_filename, pcd, write_ascii=False)

In [ ]:
# Perform coordinate transformation
transformed_filename = os.path.join(output_path, base_name + " - transformed" + extension)
coordinate_transformation(clustered_filename, transformed_filename)

## Step 2. Crop point cloud

In [ ]:
# Input point cloud file
transformed_filename = os.path.join(output_path, base_name + " - transformed" + extension)

In [ ]:
def crop(input_filename, output_filename):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    x_min, x_max = [-4.0, 8.0]
    y_min, y_max = [1e-1, math.inf]
    z_min, z_max = [-math.inf, 0.5]

    # Create bounding box
    min_bound = np.array([x_min, y_min, z_min])
    max_bound = np.array([x_max, y_max, z_max])
    bounding_box = o3d.t.geometry.AxisAlignedBoundingBox(min_bound, max_bound)

    # Crop the point cloud
    pcd = pcd.crop(bounding_box)

    o3d.t.io.write_point_cloud(output_filename, pcd, write_ascii=False)

In [ ]:
# Perform crop point cloud
cropped_filename = os.path.join(output_path, base_name + " - cropped" + extension)
crop(transformed_filename, cropped_filename)

## Step 3. Split point cloud

In [ ]:
# Input point cloud file
cropped_filename = os.path.join(output_path, base_name + " - cropped" + extension)

In [ ]:
def hidden_points_removal(pcd):
    # Radius of the spherical projection
    diameter = np.linalg.norm(pcd.get_max_bound().numpy() - pcd.get_min_bound().numpy())
    radius = diameter * 100

    # Camera location
    camera = pcd.get_center().numpy()
    camera_height = 5
    camera[2] = camera_height  # set z

    # Get all points that are visible from given view point
    _, mask = pcd.hidden_point_removal(camera, radius)
    pcd_new = pcd.select_by_index(mask)

    return pcd_new

In [ ]:
from matplotlib import ticker


# Split the point cloud, then draw a depth image and a cross-section image on each slice
def split(input_filename, every_k_meter=10):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    x_max_global, y_max_global, z_max_global = pcd.get_max_bound().numpy()
    x_min_global, y_min_global, z_min_global = pcd.get_min_bound().numpy()

    print(f"Total length: {y_max_global:.2f} meters\n")

    y_split_pos = [0]
    while y_split_pos[-1] < y_max_global:
        y_split_pos.append(y_split_pos[-1] + every_k_meter)

    y_boundary = []
    for i in range(len(y_split_pos) - 1):
        boundary = [y_split_pos[i], y_split_pos[i + 1]]
        y_boundary.append(boundary)
    print(f"Split the point cloud into {len(y_boundary)} slices: {y_boundary}\n")

    slices_path = os.path.join(output_path, base_name + " - slices")
    os.makedirs(slices_path, exist_ok=True)

    pcd_path = os.path.join(slices_path, "point cloud")
    os.makedirs(pcd_path, exist_ok=True)

    depth_path = os.path.join(slices_path, "depth images")
    os.makedirs(depth_path, exist_ok=True)

    cross_section_path = os.path.join(slices_path, "cross-section images")
    os.makedirs(cross_section_path, exist_ok=True)

    for y_min, y_max in y_boundary:
        if [y_min, y_max] != y_boundary[0]:
            break

        min_bound = np.array([x_min_global, y_min, z_min_global])
        max_bound = np.array([x_max_global, y_max, z_max_global])
        bounding_box = o3d.t.geometry.AxisAlignedBoundingBox(min_bound, max_bound)

        pcd_slice = pcd.crop(bounding_box)

        slice_number = y_max // 10
        slice_base_name = base_name + " - slice {}".format(slice_number)

        # Extract point coordinates
        coordinates = pcd_slice.point.positions.numpy()
        x, y, z = coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]
        range_x, range_y, range_z = np.ptp(x), np.ptp(y), np.ptp(z)

        # Create a figure and define the grid layout
        fig = plt.figure(figsize=(16, 9))  # Set the overall figure size
        gs = fig.add_gridspec(3, 4)  # Define a 3x4 grid for subplots

        fig.suptitle("Slice Profile — " + slice_base_name)

        # 1. Point cloud
        ax_point = fig.add_subplot(gs[:2, :2], projection="3d")  # 3D subplot on the top left
        ax_point.set(xlabel="x axis", ylabel="y axis", zlabel="z axis", zticks=[])
        ax_point.set_box_aspect([range_x, range_y, range_z])
        ax_point.scatter3D(x, y, z, s=0.01, marker='.')

        ax_point.view_init(elev=45, azim=-115)

        # 2. Cross-section image
        ax_cross = fig.add_subplot(gs[2, :2])  # 2D subplot on the bottom left
        ax_cross.set(xlabel="x", ylabel="z", xlim=[-5.0, 9.0], ylim=[-1.0, 1.5], title="Cross-section Image")
        ax_cross.set_box_aspect(range_z / range_x)
        ax_cross.scatter(x, z, s=0.01, marker='.')

        ax_cross.xaxis.set_major_formatter('{:.1f}'.format)
        ax_cross.xaxis.set_minor_locator(ticker.AutoMinorLocator())
        z_min = pcd_slice.get_min_bound().numpy()[2]
        ax_cross.set_yticks([1.0, 0.0, z_min], ["1.00", "0.00", f"z_min: {z_min:.2f}"])

        # 3. Depth images
        ax_depth = fig.add_subplot(gs[:, 2:])  # 2D subplot on the right
        ax_depth.set(xlabel="x", ylabel="y", title="Depth Image")
        ax_depth.set_box_aspect(range_y / range_x)
        ax_depth.scatter(x, y, s=0.1, marker='.')

        ax_depth.xaxis.set_minor_locator(ticker.AutoMinorLocator())
        ax_depth.yaxis.set_minor_locator(ticker.AutoMinorLocator())

        plt.tight_layout()
        plt.show()

        # image_filename = os.path.join(slices_path, slice_base_name + ".png")
        # fig.savefig(image_filename, format="png", dpi=300)

        # Save the split point cloud
        # pcd_filename = os.path.join(pcd_path, slice_base_name + extension)
        # o3d.t.io.write_point_cloud(pcd_filename, pcd_slice, write_ascii=False)

In [ ]:
# Perform split point cloud
split(cropped_filename)